# Dataset Preprocessing
Preprocessing images and bboxes before creating models.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import cv2
from torchvision import transforms
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import torchvision.transforms as T
import pandas as pd
from matplotlib import pyplot as plt
from skimage.feature import hog
import glob
from skimage import exposure
from PIL import Image, ImageDraw
import csv
import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision
from torchvision.ops import nms
sns.set(font_scale=1.3)


First, we define a preprocessor class to prepare data considering every image has 3 components: image, bounding box and label. For each image in both train and validation dataset there are 4 coordinates, a label and the image iteself.

In [111]:
class Preprocessor:
    def __init__(self, relative_path='DATASET PATH CONTAINIG ALL IMAGES AND BOUNDING BOXES AND LABELS'):
        self.current_path = os.getcwd().replace('\\', '/')
        self.base_path = self.current_path + relative_path

    def _get_filenames(self, directory, extension):
        return [
            self.base_path + directory + filename for filename in os.listdir(self.base_path + directory)
            if filename.split(".")[1:] == extension.split('.')
        ]

    @staticmethod
    def get_annotations_data(image_filenames, box_filenames):
        annotations_data = []
        for index, box_filename in enumerate(box_filenames):
            with open(box_filename, 'r') as box_file:
                box_data = box_file.read()
                rows = box_data.split('\n')
                for i in range(0, len(rows) - 1):
                    annotations_data.append(
                        [image_filenames[index]] + [int(x) for x in rows[i].split('\t')] + ['LABEL SHOULD BE HERE']
                    )
        return annotations_data

    def save_csv_file(self, filename, data):
        with open(self.current_path + '/' + filename, 'w', newline='') as csv_file:
            wr = csv.writer(csv_file)
            wr.writerows(data)

    def get_train_data(self, directory='train/', image_extension='jpg', box_extension='bboxes.tsv'):
        image_filenames = self._get_filenames(directory, image_extension)
        box_filenames = self._get_filenames(directory, box_extension)
        return image_filenames, box_filenames

    def get_test_data(self, directory='validation/', image_extension='jpg', box_extension='bboxes.tsv'):
        image_filenames = self._get_filenames(directory, image_extension)
        box_filenames = self._get_filenames(directory, box_extension)
        return image_filenames, box_filenames

In [112]:
preprocessor = Preprocessor()

Preparing train and test

In [113]:
train_image_filenames, train_box_filenames = preprocessor.get_train_data()
test_image_filenames, test_box_filenames = preprocessor.get_test_data()
train_data = preprocessor.get_annotations_data(train_image_filenames, train_box_filenames)
test_data = preprocessor.get_annotations_data(test_image_filenames, test_box_filenames)

Creating 2 dataframes for train annotations, test annotations 

In [114]:
preprocessor.save_csv_file('train_annotations.csv', train_data)
preprocessor.save_csv_file('validation_annotations.csv', test_data)

Now for each image, we have a directory, x, y, w, h which are bounding box's features and the label